<p>The purpose of step 2 is to create values for each InfoGroup record based on the OMB's Urban Area concept.</p>

In [ ]:
import pandas as pd
import geopandas as gpd

import warnings
warnings.filterwarnings("ignore")

In [ ]:
tract_df = pd.read_csv('/InfoGroup/data/rurality/reference/geographical/points-in-polygons/data/all_tracts.csv',
                 dtype=object)
tract_gdf = gpd.GeoDataFrame(tract_df)

In [ ]:
yr = 2017
df = pd.read_csv(f'/InfoGroup/data/rurality/step1_{yr}.csv',dtype=object)

In [ ]:
tract_df = pd.DataFrame(tract_gdf[['GEOID','UA_GEOID10','UATYP10','rural_tract']],dtype=object)
tract_df.rename(columns={'UA_GEOID10':'UA Code','UATYP10':'UA Type'},inplace=True)

In [ ]:
merged = df.merge(tract_df,how='inner',left_on='Full Census Tract',right_on='GEOID',indicator=True)

-----------

validation

-----------

In [ ]:
merged['_merge'].value_counts()

In [ ]:
len(merged[merged['UA Code'].isnull()])

In [ ]:
merged['rural_tract'].value_counts()

In [ ]:
merged['rural_tract'].value_counts(normalize=True) * 100

--------------

end of validation

--------------

In [ ]:
merged.drop(columns=['_merge','GEOID'],inplace=True)

In [ ]:
merged.to_csv(f'/InfoGroup/data/rurality/step2_{yr}.csv',index=None)